In [2]:
import matplotlib

In [1]:
#
# Revisar manualmente as imagens em C:\mestrado\datasets\samples_font  ok
#
#
# Pegar todas as imagens em final-32K-8lines do tipo sync_**.jpg e sobrescrever
# usando como texto a sequencia no id correspondente em portugues.
#
# Para os fontes carregar a lista de fontes cujas imagens existem em samples_font
# buscando as fontes em C:/fonts, mais as fontes do windows usado anteriormente
# Usar as fontes aleatoriamente...
#


In [4]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np  
import os
import time
import json
import random
import pickle
import cv2
import os.path
from os import path
import shutil
from glob import glob
from shutil import copyfile
from pathlib import Path
from matplotlib import pyplot as plt
from IPython.display import clear_output, display
from collections import Counter
import builtins

from sklearn.utils import shuffle

#
# gera imagem com jogadas escritas com fonte manuscrito
#
# -*- coding: utf-8 -*-
import PIL
from PIL import ImageFont
from PIL import Image, ImageDraw
from PIL import ImageColor

from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter

# print(tf.__version__) 

In [5]:
def read_label( path):
  f= open( path)
  d= f.read()
  f.close()
  return d 

def write_label( path, text):
  f= open( path, "w")
  f.write( text)
  f.close()

def show( img, name= 'noname'):
  plt.imshow( img)
  plt.title( name)
  plt.show()

def show_gimage( img, name= 'noname'):
  plt.imshow( img, cmap= 'gray')
  plt.title( name)
  plt.show()

def _load_image( path):
  # return cv2.imread( path, cv2.IMREAD_GRAYSCALE)
  return cv2.imread( path)
  
def write_image( path, img):
  cv2.imwrite( path, img)

def resize( img, scale_percent= 60, verbose=False):
  #scale_percent = 60 # percent of original size
  width = int(img.shape[1] * scale_percent / 100)
  height = int(img.shape[0] * scale_percent / 100)
  dim = (width, height)
  if verbose:
    print( "scale", scale_percent)
    print( "dim", dim)
  # resize image
  return cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 

In [23]:

# https://gist.github.com/erniejunior/601cdf56d2b424757de5
def elastic_transform_2(image, alpha, sigma,random_state=None):
#     print( alpha, sigma)
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dz = np.zeros_like(dx)

#     x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))
    x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
#     print (x.shape)
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    distored_image = map_coordinates(image, indices, order=1, mode='reflect')
    return distored_image.reshape(image.shape)

In [24]:
def transform( img):
    # ELASTIC_ALPHA_SIGMA = ((1201, 10), (1501, 12), (991, 8))
    if type( img)== PIL.Image.Image:
        img= np.asarray( img)
        img= elastic_transform_2(img, 1201, 10)  #random.randint( 991, 1501), random.randint( 8, 12))
        img= PIL.Image.fromarray( img)
        return img
    else:
        return elastic_transform_2(img, 1201, 10)  #random.randint( 991, 1501), random.randint( 8, 12))
    

In [10]:
# pgnfile= "C:/mestrado/datasets/handmade/imagens-originais/pgn_30k_pt.txt"
# with open(pgnfile) as f:
#     lines = [line.rstrip() for line in f]
    
# lines= [re.sub("[\[].*?[\]]", "", ln) for ln in lines]
# lines= [ln[8:] for ln in lines]
# lines= [ln.split()[0:32] for ln in lines]

# pgnfile= "C:/mestrado/datasets/handmade/imagens-originais/pgn_30k_eng.txt"
# with open(pgnfile) as f:
#     en_lines = [line.rstrip() for line in f]
    
# en_lines= [re.sub("[\[].*?[\]]", "", ln) for ln in en_lines]
# en_lines= [ln[8:] for ln in en_lines]
# en_lines= [ln.split()[0:32] for ln in en_lines]

pgnfile= "C:/mestrado/datasets/jogos/pgn_30k_pt.txt"
with open(pgnfile) as f:
    lines = [line.rstrip() for line in f]
    
lines= [re.sub("[\[].*?[\]]", "", ln) for ln in lines]
lines= [ln[8:] for ln in lines]
lines= [ln.split() for ln in lines]

en_pgnfile= "C:/mestrado/datasets/jogos/pgn_30k_eng.txt"
with open(en_pgnfile) as f:
    en_lines = [line.rstrip() for line in f]
    
en_lines= [re.sub("[\[].*?[\]]", "", ln) for ln in en_lines]
en_lines= [ln[8:] for ln in en_lines]
en_lines= [ln.split() for ln in en_lines]

In [25]:
print( lines[0], en_lines[0])

['d4', 'f5', 'Cf3', 'g6', 'g3', 'Bg7', 'Bg2', 'Cf6', 'b4', 'c6', 'c4', 'd5', 'Db3', 'Be6', 'c5', 'Ce4', 'O-O', 'Cd7', 'Bf4', 'O-O', 'Cbd2', 'Bf7', 'De3', 'De8', 'Bh6', 'Cdf6', 'Ce5', 'Cxd2', 'Dxd2', 'Cd7', 'Bxg7', 'Rxg7', 'Df4', 'Be6', 'a4', 'Cxe5', 'Dxe5+', 'Tf6', 'f4', 'Rg8', 'Tfb1', 'Bc8', 'Dc7', 'Dd7', 'Dxd7', 'Bxd7', 'b5', 'Tff8', 'e3', 'e6', 'Bf1', 'Rf7', 'a5', 'Tfb8', 'Tb2', 'Re7', 'Tab1', 'Tc8', 'bxc6', 'bxc6', 'Ba6', 'Tc7', 'Tb8', 'Txb8', 'Txb8', 'Be8', 'Bb7', 'Bd7', 'a6', 'Be8', 'Ta8', 'Bd7', 'Rf2', 'Rf6', 'Th8', 'Rg7', 'Ta8', 'Rf6', 'Re2', 'h6', 'Rd2', 'g5', 'Rc3', 'gxf4', 'exf4'] ['d4', 'f5', 'Nf3', 'g6', 'g3', 'Bg7', 'Bg2', 'Nf6', 'b4', 'c6', 'c4', 'd5', 'Qb3', 'Be6', 'c5', 'Ne4', 'O-O', 'Nd7', 'Bf4', 'O-O', 'Nbd2', 'Bf7', 'Qe3', 'Qe8', 'Bh6', 'Ndf6', 'Ne5', 'Nxd2', 'Qxd2', 'Nd7', 'Bxg7', 'Kxg7', 'Qf4', 'Be6', 'a4', 'Nxe5', 'Qxe5+', 'Rf6', 'f4', 'Kg8', 'Rfb1', 'Bc8', 'Qc7', 'Qd7', 'Qxd7', 'Bxd7', 'b5', 'Rff8', 'e3', 'e6', 'Bf1', 'Kf7', 'a5', 'Rfb8', 'Rb2', 'Ke7', 'Rab1', '

In [26]:
print( len( lines), len(en_lines))

30000 30000


In [ ]:
#
# le todos os fontes
#

In [27]:
new_fonts_files= glob( 'C:/mestrado/datasets/samples_font/*.ttf.jpg')

In [28]:
print( len( new_fonts_files))

77


In [29]:
new_fonts_path= [
    os.path.join( 'C:/mestrado/datasets/fonts', 
                 Path( f).name.replace( '.jpg', '')) for f in new_fonts_files]

In [31]:
new_fonts= [ImageFont.truetype(f, 30) for f in new_fonts_path]

In [32]:
fonts= [
    ImageFont.truetype('c:/Windows/Fonts/BRADHITC.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/ITCKRIST.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/LHANDW.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/Inkfree.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/OCRAEXT.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/TEMPSITC.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/PAPYRUS.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/MAIAN.TTF', 30)
] 

In [33]:
all_fonts= new_fonts + fonts

In [34]:
print( len( new_fonts), len( fonts), len( all_fonts))

77 8 85


In [35]:
Path( new_fonts_path[0]).name

'aescrawl.ttf'

In [21]:
#
# le todos os arquivos cuja imagem vai sobrescrever
#
# train_label_files= glob(
#     os.path.join( 'C:/mestrado/datasets/unified/final-32K-8lines--v4',
#                  'train/labels/syn*.pgn'))

# valid_label_files= glob(  
#     os.path.join( 'C:/mestrado/datasets/unified/final-32K-8lines--v4',
#                  'valid/labels/syn*.pgn'))

# label_files= train_label_files+ valid_label_files

In [22]:
# len( label_files)

25641

In [50]:
#
# recupera o label original de cada arquivo
#

6055

In [19]:
# int( Path( label_files[0]).name[4:9])

49

In [23]:
# def labelof( f):
#     id= int( Path( f).name[4:9])
# #     print( id)
#     return lines[ id-1]

In [24]:
# i= 100
# print( 'labels of ', label_files[i], ' '.join( labelof( label_files[i])[0:16]))

labels of  C:/mestrado/datasets/unified/final-32K-8lines--v4\train/labels\syn_00995.pgn e4 e6 d4 d5 Cd2 Cf6 e5 Cfd7 Bd3 c5 c3 Cc6 Ce2 cxd4 cxd4 f6


In [ ]:
# # abre o arquivo original
# img = Image.open("C:/mestrado/datasets/syn/template/template.jpg")  # load base image
# dctx = ImageDraw.Draw(img)  # create drawing context

In [ ]:
# show( img)

In [ ]:
# fnt= ImageFont.truetype('c:/fonts/GOODDP__.TTF', 30)

In [643]:

# def load_fonts():
#     files= glob( "C:/fonts/*.ttf")
#     fonts= []
#     for f in files:
#         fnt= ImageFont.truetype(f, 30)
#         fonts.append( (f, fnt))
#     return fonts
        
    
# all_fonts= load_fonts()    
        
# for name,fnt in all_fonts:    
#     img = Image.open("C:/mestrado/datasets/syn/template/template.jpg")  # load base image
#     dctx = ImageDraw.Draw(img)  # create drawing context

#     line= "CBxd4+"

#     dh_top= 7
#     dh_bottom= 13
#     h= (img.height - dh_top - dh_bottom)/25.0
#     w= img.width/4.0

#     txtsz = dctx.textsize(line, fnt)
#     x= (w - txtsz[0])/2
#     y= 7 + (h - txtsz[1])/2 #+ margin_top
#     dctx.text( (x, y), line, font=fnt, fill="#000000")

#     imgc= img.crop( (0, 0, w, h+7*2))
    
#     imgfile= os.path.join( "C:/mestrado/datasets/samples_font", Path(name).name+".jpg")
#     print( 'save to ', imgfile)
#     imgc.save( imgfile)    


In [ ]:
# showbig( img)

In [36]:
def showbig( img, name= 'noname'):
    plt.figure(figsize = (10,10))    
    plt.imshow( img)
    plt.title( name)
    plt.show()

In [ ]:
# import glob

In [ ]:
# #
# #  gerar somente os que nao tem no handwritten
# #

# # carrega os jogos existentes
# handfiles= glob.glob( "C:/mestrado/datasets/unified/parts/handwritten/labels/*.pgn")

In [ ]:
# print( len( handfiles))

In [ ]:
# fname= handfiles[0]


In [ ]:
# Path(fname).name.split('_')

In [ ]:
# # a3__d5__d4__Nf6_5742_1494_R_part3_scan0145
# id= int( Path(fname).name.split('_')[-5])-1
# print( id)

In [ ]:
# print( read_label( fname))

In [ ]:
# " ".join( en_lines[ id])

In [ ]:
# for f in handfiles:
#     if 'scan' not in Path(f).name:
#         print( Path(f).name)


In [ ]:
# s= set()
# for f in handfiles:
#     s.add( int( Path(f).name.split('_')[-5][-4:])-1)

In [ ]:
# print( len( s))

In [ ]:
# #
# # filtra os que tem manual, os que tem tamanho menor de 16
# #
# new_lines= []
# new_en_lines=[]
# new_no= []

# for i in range( 0, len( lines)):
#     if len( en_lines[i])< 32:
#         continue
#     if i in s:
#         continue
#     new_lines.append( lines[i]) 
#     new_en_lines.append( en_lines[i]) 
#     new_no.append( i)    

In [ ]:
# print( len( new_lines))
# print( len( s))
# print( len( lines))

In [ ]:
# print( new_no[0:10])

In [ ]:
# #
# # separa em valid e test
# #
# n= len( new_lines)

# # precisa fazer shuffle?

# n_train= int(n*0.8)
# new_train_lines= new_lines[:n_train]
# new_train_en_lines=new_en_lines[:n_train]
# new_train_no= new_no[:n_train]

# new_valid_lines= new_lines[n_train:]
# new_valid_en_lines=new_en_lines[n_train:]
# new_valid_no= new_no[n_train:]

In [ ]:
# print( n, n_train)

In [37]:
len( all_fonts)

85

In [38]:
def sp_noise(npimage,prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = np.zeros(npimage.shape,np.uint8)
    thres = 1 - prob 
    for i in range(npimage.shape[0]):
        for j in range(npimage.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = random.randint( 0, 128)
            else:
                output[i][j] = npimage[i][j]
    return output

def sp_noise_pil( pilimg, prob):
#     print( 'noise', prob)
    img= pilimg
    img= np.asarray( img)
    img= sp_noise(img, prob)
    img= PIL.Image.fromarray( img)
    return img

def distorce( img):
    # ELASTIC_ALPHA_SIGMA = ((1201, 10), (1501, 12), (991, 8))
#     print( 'distorce')
    if type( img)== PIL.Image.Image:
        img= np.asarray( img)
        img= elastic_transform_2(img, 991, 8)  #random.randint( 991, 1501), random.randint( 8, 12))
        img= PIL.Image.fromarray( img)
        return img
    else:
        return elastic_transform_2(img, 991, 8)  #random.randint( 991, 1501), random.randint( 8, 12))
    

# Rotate the foreground
def rotate( pilimg):
#     print( 'rotate')
    foreground= pilimg
    angle_degrees = random.uniform(-2.5,2.5)
#     print( angle_degrees)
    foreground = foreground.rotate(angle_degrees, resample=Image.BICUBIC, expand=True, fillcolor = (255,255,255))

    # Scale the foreground
    scale = random.random() * .5 + .5 # Pick something between .5 and 1
    new_size = (int(foreground.size[0] * scale), int(foreground.size[1] * scale))
    foreground = foreground.resize(new_size, resample=Image.BICUBIC)
    return foreground
#     show( foreground)
    

def random_transform( pilimg, p):
    img= pilimg
    if( random.random()< 0.8):
        img= rotate( img)
#     if( random.random()< 0.1):
#         img= distorce( img)
    if( random.random()< 0.25):
        img= sp_noise_pil( img, random.uniform( 0.001, 0.0001))
    
    return img


In [12]:


# 
# le o arquivo
#



#
# conta ate 16
#
flat_tokens16 = [t for _list in lines for t in _list[0:16]]
print( 'len( flat)', len( flat_tokens16))
count16= Counter( flat_tokens16)
print( 'len( count16)', len( count16))  #esperado 776
counted16= {k: v for k, v in sorted(count16.items(), key=lambda item: item[1], reverse=True)}
# print( counted16)   

#
# gerar imagens usando portugues, mas o label correspondente em ingles
#

len( flat) 473529
len( count16) 776


In [75]:

# 
# transforma de portugues para ingles, para gerar pgn

def pt_to_pgn( w):
    # R=> K
    # T=> R
    # C=> N
    # B=> B
    # D=> Q

#     print( w, '=> ')
    png= w.replace( 'R', 'K').replace( 'T', 'R').replace( 'C', 'N').replace( 'D', 'Q')
#     print( png)
    return png

def to_pgn( label):
    return [pt_to_pgn( w) for w in label]


In [76]:
print( en_lines[0])
print( [pt_to_pgn( t) for t in lines[0]])
print( to_pgn( lines[0]))

['d4', 'f5', 'Nf3', 'g6', 'g3', 'Bg7', 'Bg2', 'Nf6', 'b4', 'c6', 'c4', 'd5', 'Qb3', 'Be6', 'c5', 'Ne4', 'O-O', 'Nd7', 'Bf4', 'O-O', 'Nbd2', 'Bf7', 'Qe3', 'Qe8', 'Bh6', 'Ndf6', 'Ne5', 'Nxd2', 'Qxd2', 'Nd7', 'Bxg7', 'Kxg7', 'Qf4', 'Be6', 'a4', 'Nxe5', 'Qxe5+', 'Rf6', 'f4', 'Kg8', 'Rfb1', 'Bc8', 'Qc7', 'Qd7', 'Qxd7', 'Bxd7', 'b5', 'Rff8', 'e3', 'e6', 'Bf1', 'Kf7', 'a5', 'Rfb8', 'Rb2', 'Ke7', 'Rab1', 'Rc8', 'bxc6', 'bxc6', 'Ba6', 'Rc7', 'Rb8', 'Rxb8', 'Rxb8', 'Be8', 'Bb7', 'Bd7', 'a6', 'Be8', 'Ra8', 'Bd7', 'Kf2', 'Kf6', 'Rh8', 'Kg7', 'Ra8', 'Kf6', 'Ke2', 'h6', 'Kd2', 'g5', 'Kc3', 'gxf4', 'exf4']
['d4', 'f5', 'Nf3', 'g6', 'g3', 'Bg7', 'Bg2', 'Nf6', 'b4', 'c6', 'c4', 'd5', 'Qb3', 'Be6', 'c5', 'Ne4', 'O-O', 'Nd7', 'Bf4', 'O-O', 'Nbd2', 'Bf7', 'Qe3', 'Qe8', 'Bh6', 'Ndf6', 'Ne5', 'Nxd2', 'Qxd2', 'Nd7', 'Bxg7', 'Kxg7', 'Qf4', 'Be6', 'a4', 'Nxe5', 'Qxe5+', 'Rf6', 'f4', 'Kg8', 'Rfb1', 'Bc8', 'Qc7', 'Qd7', 'Qxd7', 'Bxd7', 'b5', 'Rff8', 'e3', 'e6', 'Bf1', 'Kf7', 'a5', 'Rfb8', 'Rb2', 'Ke7', 'Rab1', '

In [15]:
" ".join( en_lines[0])== " ".join( [pt_to_png( t) for t in lines[0]])

True

In [19]:
count16

Counter({'d4': 23906,
         'f5': 1633,
         'Cf3': 24462,
         'g6': 9536,
         'g3': 6466,
         'Bg7': 8945,
         'Bg2': 6088,
         'Cf6': 24489,
         'b4': 787,
         'c6': 6719,
         'c4': 13249,
         'd5': 15863,
         'Db3': 714,
         'Be6': 1073,
         'c5': 13768,
         'Ce4': 1106,
         'e4': 19723,
         'd6': 12986,
         'cxd4': 7311,
         'Cxd4': 5321,
         'Cc3': 19474,
         'a6': 5914,
         'Bc4': 3185,
         'e6': 14668,
         'Bb3': 1261,
         'b5': 2694,
         'O-O': 25033,
         'Be7': 7379,
         'e3': 5802,
         'Cbd2': 2073,
         'dxc5': 898,
         'Bxc5': 405,
         'Dc2': 1621,
         'Ba7': 72,
         'a3': 2228,
         'Cc6': 14029,
         'Be3': 3889,
         'Be2': 4566,
         'Cb8': 111,
         'a4': 1321,
         'e5': 11864,
         'Bf5': 1846,
         'c3': 4774,
         'Cge7': 1116,
         'Db6': 1293,
         'Bb5': 2

In [20]:
words= [item for item in count16]

In [22]:
print( len( words), words)

776 ['d4', 'f5', 'Cf3', 'g6', 'g3', 'Bg7', 'Bg2', 'Cf6', 'b4', 'c6', 'c4', 'd5', 'Db3', 'Be6', 'c5', 'Ce4', 'e4', 'd6', 'cxd4', 'Cxd4', 'Cc3', 'a6', 'Bc4', 'e6', 'Bb3', 'b5', 'O-O', 'Be7', 'e3', 'Cbd2', 'dxc5', 'Bxc5', 'Dc2', 'Ba7', 'a3', 'Cc6', 'Be3', 'Be2', 'Cb8', 'a4', 'e5', 'Bf5', 'c3', 'Cge7', 'Db6', 'Bb5', 'Cxe4', 'Cd6', 'Bxc6', 'dxc6', 'dxe5', 'Cf5', 'h6', 'b3', 'b6', 'Bb2', 'Bb7', 'cxd5', 'Cxd5', 'Cdb4', 'Cxc6', 'Dxd1+', 'Cb5', 'Bg5', 'Cfd7', 'Bxe7', 'Dxe7', 'Bd3', 'exd4', 'h3', 'a5', 'f4', 'Bc5', 'Cxe5', 'dxe4', 'Bxf7+', 'Rf8', 'Cg4', 'Cxg4', 'Dh4+', 'De2', 'De7', 'd3', 'Cbd7', 'Dxe2+', 'Cg5', 'exd5', 'Ca5', 'Bb5+', 'bxc6', 'Df3', 'Bb4+', 'Bd2', 'Bxc3', 'Tc1', 'h5', 'Ch6', 'Bg4', 'Ba4', 'Cc5', 'Dxd4', 'Ch3', 'Cf4', 'cxb5', 'bxa6', 'Dxd5', 'Dd4', 'f6', 'Bf4', 'Dc7', 'fxe4', 'Bd6', 'Ce5', 'Cd2', 'Bd7', 'Cd5', 'De4', 'Bxd3', 'Dxd3', 'Ce2', 'f3', 'Cge2', 'Dd2', 'Te8', 'Ce7', 'axb5', 'Cgf3', 'h4', 'Dd7', 'Tb1', 'Cb3', 'Bb4', 'Bxc3+', 'bxc3', 'De8', 'Bh4', 'Cdb5', 'Cxc3', 'Cce2', 'D

In [ ]:
count16


In [54]:
def random_label_from_vocab():
    label= []
    for i in range( 0, 16):
        label.append( words[random.randrange( 0, len( words))])
    return label


print( random_label_from_vocab())


['Dd1', 'Rxe7', 'Dxf4', 'Re7', 'Cxd4+', 'Cdc3', 'De4', 'De6+', 'a4', 'Dh6', 'Cfd4', 'Cxe2', 'Cf4+', 'Bxb7', 'cxd3', 'Th3']


In [55]:
dd= {w:0 for w in words}

In [59]:
for i in range( 0, 20000*16):
    w= words[random.randrange( 0, len( words))]
    dd[ w]= dd[ w]+1
    

In [60]:
print( dd)

{'d4': 401, 'f5': 420, 'Cf3': 433, 'g6': 447, 'g3': 444, 'Bg7': 411, 'Bg2': 426, 'Cf6': 445, 'b4': 448, 'c6': 430, 'c4': 403, 'd5': 385, 'Db3': 368, 'Be6': 411, 'c5': 406, 'Ce4': 394, 'e4': 407, 'd6': 423, 'cxd4': 421, 'Cxd4': 413, 'Cc3': 421, 'a6': 420, 'Bc4': 432, 'e6': 386, 'Bb3': 406, 'b5': 423, 'O-O': 404, 'Be7': 406, 'e3': 439, 'Cbd2': 438, 'dxc5': 403, 'Bxc5': 426, 'Dc2': 417, 'Ba7': 397, 'a3': 441, 'Cc6': 465, 'Be3': 415, 'Be2': 412, 'Cb8': 395, 'a4': 387, 'e5': 384, 'Bf5': 411, 'c3': 411, 'Cge7': 392, 'Db6': 407, 'Bb5': 418, 'Cxe4': 395, 'Cd6': 410, 'Bxc6': 401, 'dxc6': 409, 'dxe5': 362, 'Cf5': 423, 'h6': 419, 'b3': 391, 'b6': 411, 'Bb2': 409, 'Bb7': 419, 'cxd5': 427, 'Cxd5': 417, 'Cdb4': 410, 'Cxc6': 412, 'Dxd1+': 423, 'Cb5': 418, 'Bg5': 432, 'Cfd7': 421, 'Bxe7': 405, 'Dxe7': 436, 'Bd3': 409, 'exd4': 395, 'h3': 441, 'a5': 435, 'f4': 413, 'Bc5': 413, 'Cxe5': 447, 'dxe4': 406, 'Bxf7+': 418, 'Rf8': 411, 'Cg4': 432, 'Cxg4': 411, 'Dh4+': 430, 'De2': 392, 'De7': 415, 'd3': 403, 'Cb

In [62]:
min([k for (k,v) in dd])

ValueError: too many values to unpack (expected 2)

In [72]:
min( [v for (k,v) in dd.items()])

346

In [73]:
max( [v for (k,v) in dd.items()])

476

In [269]:
img = Image.open("C:/mestrado/datasets/syn/template/template.jpg")  # load base image
dctx = ImageDraw.Draw(img)  # create drawing context
dh_top= 7
dh_bottom= 13
h= (img.height - dh_top - dh_bottom)/25.0
w= img.width/4.0

margin_left= w/4
margin_top= h/6

def resize_fixed( img):
    _dd = (271, 230) #width, height
    return cv2.resize(img, _dd, interpolation = cv2.INTER_AREA) 

def generate_for( line):
#     if fnt is None:
#         fnt= random.choice( fonts)
#     print( fnt.getname())
    
    img = Image.open("C:/mestrado/datasets/syn/template/template.jpg")  # load base image
    dctx = ImageDraw.Draw(img)  # create drawing context

    i= 0
    for ln in range( 0, 8):
        for col in range( 0, 2):
            if i>= len( line):
                del dctx
                return img
        
            fnt= random.choice( fonts)
            txtsz = dctx.textsize(line[i], fnt)
        
            x= w* col + (w - txtsz[0])/2
            if random.random()< 0.7:
                xmargem= int( (w - txtsz[0])/3)
                x+= random.randint( -xmargem, xmargem)
            y= dh_top + h*ln + (h - txtsz[1])/2 #+ margin_top

            dctx.text( (x, y), line[i], font=fnt, fill="#000000")
            i= i+1
            
    del dctx 
    return img

def cut( img):
    h= 300
    w= 345

    img1= img[:h, :w]
    return img1

def generate_one( label): #pt_line, pgn, no, folder):
#     id_font= no%len( all_fonts)
    img= generate_for( label)
    img= cut( np.asarray(img))
#     img= resize_fixed( img)
    img= PIL.Image.fromarray(np.uint8(img))
#     img.save( img_file)
#     img.save( img_file.replace( '.jpg', '-'+Path( new_fonts_path[id_font]).name+'.jpg'))
    return img


def __generate():
    _files= label_files
#     print( 'Gerando ', len( _files))
    for i in range( 0, len( _files)):
        if i%100== 0:
            print( i)
        img_file= _files[i].replace( 'labels', 'images.new').replace( '.pgn', '.jpg') 
        img= generate_one( i, labelof( _files[i])[0:16], img_file)
        
        img= random_transform( img, 0.3)
        img.save( img_file)
#         print( img_file)
#         show( img__)
#         show( transform( img__))
#         return img
    
# __generate()
# img1= __generate()
# show( img1)
# pilimg= img1
# npimg= np.asarray( img1)


def __generate_random( N):
    dataset_dir= 'C:/mestrado/datasets/unified/parts/random-8lines-vocab776'
    
    for i in range( 0, N):
        if( i%100== 0):
            print( i)
        label_pt= random_label_from_vocab()
        img= generate_one( label_pt)
        img= random_transform( img, 0.3)
        pgn= to_pgn( label_pt)

        # save
#         show( img)
        img_file= os.path.join( dataset_dir, 'images', 'random-syn-'+ str(i+1).zfill( 5)+'.jpg')
        img.save( img_file)
        write_label( img_file.replace( 'images', 'labels').replace( '.jpg', '.pgn'), 
                    " ".join( to_pgn( label_pt)))

__generate_random( 30000)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [79]:
str(4).zfill( 10)

'0000000004'

In [275]:
#
# gera um arquivo com todos os labels do dataset, até o tamanho 16
#

pgn_files= glob( 'C:/mestrado/datasets/unified/random-32k-8lines--v5/*/labels/*.pgn')
print( len( pgn_files))

33300


In [280]:
all_pgn= []
for f in pgn_files:
    all_pgn.append( read_label( f))

In [290]:
all_pgn= [' '.join( p.split()[0:16]) for p in all_pgn]

In [291]:
write_label( 'C:/mestrado/datasets/unified/random-32k-8lines--v5/all-labels-.pgn', 
           '\n'.join( all_pgn))
